In [1]:
#pip install natsort
import pandas as pd
import numpy as np
import natsort # 파일 숫자 정렬용 라이브러리
import os
import librosa

## 0. MFCC 코드

In [2]:
#https://www.analyticsvidhya.com/blog/2022/03/implementing-audio-classification-project-using-deep-learning/
#https://stackoverflow.com/questions/52841335/how-can-i-pad-wav-file-to-specific-length
from librosa.util import fix_length
from librosa import load
from tqdm import tqdm

def features_extractor(file):
    #load the file (audio)
    file_name = file
    sf = 44100 # sampling frequency of wav file
    
    audio, sample_rate = librosa.load(file_name, sr=sf, mono=True) # mono=True converts stereo audio to mono
        
    #we extract mfcc'
    mfccs_features = librosa.feature.mfcc(y=audio, n_mfcc=20, sr=sf) ## --> n_mfcc : 20 ~ 50
    #print("mfccs_features",mfccs_features.shape)

    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0) ## 평균으로 출력
    
    #print("mfccs_scaled_features", mfccs_scaled_features.shape)
    return mfccs_scaled_features

## 1. feature 데이터 가공하기

In [3]:
file_list = natsort.natsorted(os.listdir("MFCC_정리/music2_audio_wav"))

extracted_features=[]
extracted_len_features=[]
file_total_len = 1 # feature_input 길이 ex). 1초, 2초

start_sec = 1 # 파일 갯수 정하기 ex 1 ~ 24
end_sec = 128
count = 0

for filename in file_list[start_sec-1:end_sec]:
    file_name = os.path.join("MFCC_정리/music2_audio_wav", filename) ## 만약에 2초마다 붙이고 싶으면
    # print(file_name)
    data = features_extractor(file_name)
    extracted_len_features = np.concatenate((extracted_len_features, data))
    count+=1
    
    if (count == file_total_len):
        extracted_features.append(extracted_len_features)
        
        extracted_len_features=[]
        count = 0

    
extracted_features_df=pd.DataFrame(extracted_features)

for column_name in extracted_features_df:
    extracted_features_df.rename(columns={column_name:"f"+str(column_name+1)},inplace=True)


## 2. skeleton 데이터 가공하기

In [4]:
skeleton = pd.read_csv('MFCC_정리/music2_hand_result.csv')

extracted_skeleton=[] ### 궁금한게 초 길이 ## 다음 순차적으로 데이터 이어 붙이기 해야할 것같음

interval = 1
start_frame = (start_sec+interval) * 30 # 실제 csv : ( row = 2부터 시작 ) // 한 칸씩 떼기
end_frame = (end_sec+interval) * 30

frame_count = 30 # 1초 = 30 frame

skeleton_col = ['INDEX_FINGER_PIP_LX'] # 원하는 column 에 대해서만 ex. 6번 = INDEX_FINGER_PIP_LX

for frame_num in range(start_frame, end_frame + 1, frame_count): # start_frame ~ end_frame
    extracted_total_skeleton=[]
    for col in skeleton_col:
        extracted_skeleton_list = list(skeleton.loc[frame_num:(frame_num+frame_count)-1, col])
        extracted_total_skeleton = np.concatenate((extracted_total_skeleton, extracted_skeleton_list))
    extracted_skeleton.append(extracted_total_skeleton)

extracted_skeleton_df = pd.DataFrame(extracted_skeleton)

for column_name in extracted_skeleton_df:
    extracted_skeleton_df.rename(columns={column_name:"s"+str(column_name+1)},inplace=True)

In [5]:
total_data = pd.concat([extracted_features_df, extracted_skeleton_df],axis=1)
total_data

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,s21,s22,s23,s24,s25,s26,s27,s28,s29,s30
0,-659.597412,40.892715,-6.337469,2.135688,3.548059,0.804762,2.512327,-0.336857,-0.549686,0.215782,...,592.918129,942.739029,940.774612,940.764618,940.587616,941.214981,940.151138,455.353317,445.687714,442.435913
1,-292.834778,267.615631,-31.524200,-17.887941,27.018032,-12.516393,-10.344816,-1.591869,-14.941175,-10.007528,...,283.276501,291.220093,939.682617,387.712860,391.114121,395.178032,402.753715,415.230331,938.820419,939.344635
2,-305.223053,244.672012,-3.893402,-44.606812,3.250715,5.895392,1.689971,6.827463,0.744675,-5.771279,...,761.944961,1036.661758,1045.031281,1042.934799,807.675247,817.910080,822.041473,824.335251,1035.299988,1039.114380
3,-315.512512,256.956909,-1.200694,-25.436708,17.819677,-2.204413,-6.866660,6.551411,-3.638557,-9.829278,...,1038.767624,1037.162170,1037.543640,456.214523,441.257286,424.943085,1037.814865,404.944076,374.267235,360.795555
4,-314.082062,263.250031,-4.278464,-26.831131,26.338915,7.504233,-6.451949,3.456089,0.358805,-0.746851,...,1049.926910,1049.134064,1049.563370,1048.102646,400.252533,1047.496033,1048.062973,1047.405014,1049.822540,1051.230774
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,-274.375885,247.054626,-23.015581,-3.219867,10.029488,-18.128145,0.526839,0.349822,-9.175799,-7.860695,...,988.137588,1107.919617,979.821243,969.614029,963.125839,959.751892,961.448669,952.328186,950.224533,950.477905
124,-267.205414,259.817535,-30.687019,-12.229400,28.597347,-4.391084,3.123246,10.407693,-4.229204,-4.573333,...,936.883011,938.682327,936.265183,936.201553,933.104782,931.755600,932.228622,927.540894,926.596451,924.910507
125,-399.051239,292.382385,-0.795051,-45.920994,19.528372,10.774570,-9.688483,8.796929,10.584660,-4.897935,...,893.683701,890.424194,890.750885,889.465103,885.422134,883.896408,880.801086,0.000000,0.000000,0.000000
126,-486.915405,290.128601,45.051098,-47.019325,-5.759301,19.710251,3.713731,1.178202,7.826465,0.669808,...,0.000000,0.000000,722.741318,721.797485,722.536621,723.346710,0.000000,725.435028,726.107483,726.412353


In [6]:
total_data.to_csv("music2_combine.csv")

## 3. Train, Test 데이터 생성

In [7]:
import pandas as pd
total_data = pd.read_csv('music2_combine.csv')

In [8]:
total_data.drop('Unnamed: 0', inplace=True, axis=1)

In [9]:
f_list = []
s_list = []
for col in total_data.columns:
    if (col.find("f")!=-1):
        f_list.append(col)
    else:
        s_list.append(col)

In [10]:
total_data[f_list]

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20
0,-659.597412,40.892715,-6.337469,2.135688,3.548059,0.804762,2.512327,-0.336857,-0.549686,0.215782,1.039396,1.698405,0.474671,-1.070947,-1.314891,-0.188363,-0.083124,0.201764,0.626399,-0.417951
1,-292.834778,267.615631,-31.524200,-17.887941,27.018032,-12.516393,-10.344816,-1.591869,-14.941175,-10.007528,-5.137794,-6.208638,-1.840299,-5.119336,-8.581604,-5.596979,-7.949373,-7.917210,-1.329109,-0.840493
2,-305.223053,244.672012,-3.893402,-44.606812,3.250715,5.895392,1.689971,6.827463,0.744675,-5.771279,-6.333980,-2.251126,5.010935,2.434026,-7.714496,-6.270520,1.998515,0.632427,-5.379352,-6.326122
3,-315.512512,256.956909,-1.200694,-25.436708,17.819677,-2.204413,-6.866660,6.551411,-3.638557,-9.829278,-2.312829,-0.853408,-0.859357,3.551487,2.857303,-3.297808,-6.639400,-6.634170,-5.738685,-3.462696
4,-314.082062,263.250031,-4.278464,-26.831131,26.338915,7.504233,-6.451949,3.456089,0.358805,-0.746851,4.305096,1.309869,-1.735513,0.219861,-0.719799,-1.325379,3.927995,7.164885,2.745458,-2.265963
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,-274.375885,247.054626,-23.015581,-3.219867,10.029488,-18.128145,0.526839,0.349822,-9.175799,-7.860695,-9.392933,-8.148173,-0.438686,7.919489,2.013604,-5.192632,-4.235599,-4.485144,0.216761,0.204451
124,-267.205414,259.817535,-30.687019,-12.229400,28.597347,-4.391084,3.123246,10.407693,-4.229204,-4.573333,-0.256944,-1.163530,2.427042,5.000734,0.297901,-4.059734,-7.076856,-1.363354,6.904899,-1.689761
125,-399.051239,292.382385,-0.795051,-45.920994,19.528372,10.774570,-9.688483,8.796929,10.584660,-4.897935,-0.967777,6.988903,1.519527,-0.161918,5.750542,2.316281,-5.852559,-0.523886,7.681949,1.245867
126,-486.915405,290.128601,45.051098,-47.019325,-5.759301,19.710251,3.713731,1.178202,7.826465,0.669808,-2.894178,6.526373,10.251746,5.296264,3.346136,2.467661,-0.569632,0.249553,2.797758,0.480999


In [11]:
def train_test_normalize(total_data):
    # create training and test set 
    
    input = total_data[f_list]
    output = total_data[s_list]
    
    ### Train Test Split
    from sklearn.model_selection import train_test_split
    X_train,X_test,y_train,y_test=train_test_split(input,output,test_size=0.2,random_state=7)
    
    print(X_train.shape)
    print(y_train.shape)
    print(X_test.shape)
    print(y_test.shape)
    
    
    return X_train, y_train , X_test, y_test

In [18]:

X_train, y_train, X_test, y_test = train_test_normalize(total_data)
#print("X_train : ",X_train.shape, " y_train : ", y_train.shape)
#print("X_test : ", X_test.shape, " y_test : ", y_test.shape)

X_train=X_train.to_numpy()
X_test=X_test.to_numpy()



X_train = X_train.reshape(X_train.shape[0], 1, 20)
X_test = X_test.reshape(X_test.shape[0], 1, 20)

print(X_train.shape)

'''
X_train=X_train.to_numpy()
X_train = X_train.reshape(1, 102, 20)
print(X_train.shape)
y_train=y_train.to_numpy()
y_train = y_train.reshape(1, 102, 30)
print(y_train.shape)
'''

(102, 20)
(102, 30)
(26, 20)
(26, 30)
(102, 1, 20)


'\nX_train=X_train.to_numpy()\nX_train = X_train.reshape(1, 102, 20)\nprint(X_train.shape)\ny_train=y_train.to_numpy()\ny_train = y_train.reshape(1, 102, 30)\nprint(y_train.shape)\n'

In [17]:
print(type(X_train))
print(X_train)

<class 'numpy.ndarray'>
[[[-292.94378662  249.5737915    -4.20794773 ...   -2.61794829
     -2.15530825   -4.29333401]]

 [[-321.82446289  255.09492493   -9.50197983 ...  -10.15995884
     -7.42558289   -2.50874543]]

 [[-350.71487427  224.51725769    3.32479334 ...    8.09391594
     -1.1235944    -9.15687943]]

 ...

 [[-495.46795654  249.40878296   28.04263687 ...   -1.69272137
     -0.76628536   -4.88392162]]

 [[-275.90155029  259.84680176  -18.84955978 ...    0.88478196
      1.40320194   -4.00582504]]

 [[-291.63879395  244.11590576  -23.75601959 ...   -2.22315431
     -1.69253981   -8.48329926]]]


In [ ]:
#import matplotlib.pyplot as plt
#plt.plot([1,4,7,8,13,17])

In [ ]:
from numpy import array

data = array([
	[0.1, 1.0],
	[0.2, 0.9],
	[0.3, 0.8],
	[0.4, 0.7],
	[0.5, 0.6],
	[0.6, 0.5],
	[0.7, 0.4],
	[0.8, 0.3],
	[0.9, 0.2],
	[1.0, 0.1]])

print(data.shape)

data = data.reshape(1, 10, 2)
print(data.shape)

print(data)

In [23]:
from keras.layers import LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation
from keras import optimizers
from keras import metrics
from sklearn.metrics import explained_variance_score



model = Sequential()
model.add(LSTM(20, activation='relu', return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(20, activation='relu'))
model.add(Dense(30))
model.compile(loss='mae', optimizer='adam')
model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 1, 20)             3280      
                                                                 
 lstm_5 (LSTM)               (None, 20)                3280      
                                                                 
 dense_1 (Dense)             (None, 30)                630       
                                                                 
Total params: 7,190
Trainable params: 7,190
Non-trainable params: 0
_________________________________________________________________


In [31]:
from tensorflow.keras.callbacks import EarlyStopping


# model training
batch = 512
epoch = 10000
inv_train, yhat, inv_yhat, inv_y = 0, 0, 0, 0

early_stop = EarlyStopping(monitor='loss', patience=30, verbose=1)
history = model.fit(X_train, y_train, epochs=epoch, batch_size=batch,  verbose=1, callbacks=[early_stop])


    

Epoch 1/10000
1/1 [==============================] - 0s 4ms/step - loss: 181.3887
Epoch 2/10000
1/1 [==============================] - 0s 2ms/step - loss: 181.3675
Epoch 3/10000
1/1 [==============================] - 0s 3ms/step - loss: 181.3601
Epoch 4/10000
1/1 [==============================] - 0s 3ms/step - loss: 181.3595
Epoch 5/10000
1/1 [==============================] - 0s 2ms/step - loss: 181.3370
Epoch 6/10000
1/1 [==============================] - 0s 2ms/step - loss: 181.3270
Epoch 7/10000
1/1 [==============================] - 0s 3ms/step - loss: 181.3275
Epoch 8/10000
1/1 [==============================] - 0s 3ms/step - loss: 181.3096
Epoch 9/10000
1/1 [==============================] - 0s 3ms/step - loss: 181.2953
Epoch 10/10000
1/1 [==============================] - 0s 2ms/step - loss: 181.2870
Epoch 11/10000
1/1 [==============================] - 0s 3ms/step - loss: 181.2845
Epoch 12/10000
1/1 [==============================] - 0s 4ms/step - loss: 181.2761
Epoch 13/1000

1/1 [==============================] - 0s 2ms/step - loss: 179.5006
Epoch 197/10000
1/1 [==============================] - 0s 3ms/step - loss: 179.4936
Epoch 198/10000
1/1 [==============================] - 0s 2ms/step - loss: 179.4807
Epoch 199/10000
1/1 [==============================] - 0s 2ms/step - loss: 179.4753
Epoch 200/10000
1/1 [==============================] - 0s 2ms/step - loss: 179.4624
Epoch 201/10000
1/1 [==============================] - 0s 2ms/step - loss: 179.4518
Epoch 202/10000
1/1 [==============================] - 0s 2ms/step - loss: 179.4435
Epoch 203/10000
1/1 [==============================] - 0s 3ms/step - loss: 179.4342
Epoch 204/10000
1/1 [==============================] - 0s 2ms/step - loss: 179.4252
Epoch 205/10000
1/1 [==============================] - 0s 2ms/step - loss: 179.4164
Epoch 206/10000
1/1 [==============================] - 0s 2ms/step - loss: 179.4079
Epoch 207/10000
1/1 [==============================] - 0s 2ms/step - loss: 179.3979
Epoch 20

1/1 [==============================] - 0s 3ms/step - loss: 177.8254
Epoch 391/10000
1/1 [==============================] - 0s 2ms/step - loss: 177.8163
Epoch 392/10000
1/1 [==============================] - 0s 2ms/step - loss: 177.8105
Epoch 393/10000
1/1 [==============================] - 0s 2ms/step - loss: 177.8083
Epoch 394/10000
1/1 [==============================] - 0s 3ms/step - loss: 177.7910
Epoch 395/10000
1/1 [==============================] - 0s 2ms/step - loss: 177.7851
Epoch 396/10000
1/1 [==============================] - 0s 2ms/step - loss: 177.7829
Epoch 397/10000
1/1 [==============================] - 0s 2ms/step - loss: 177.7757
Epoch 398/10000
1/1 [==============================] - 0s 2ms/step - loss: 177.7630
Epoch 399/10000
1/1 [==============================] - 0s 2ms/step - loss: 177.7543
Epoch 400/10000
1/1 [==============================] - 0s 2ms/step - loss: 177.7532
Epoch 401/10000
1/1 [==============================] - 0s 2ms/step - loss: 177.7376
Epoch 40

1/1 [==============================] - 0s 2ms/step - loss: 176.2358
Epoch 585/10000
1/1 [==============================] - 0s 2ms/step - loss: 176.2386
Epoch 586/10000
1/1 [==============================] - 0s 3ms/step - loss: 176.2110
Epoch 587/10000
1/1 [==============================] - 0s 2ms/step - loss: 176.2502
Epoch 588/10000
1/1 [==============================] - 0s 3ms/step - loss: 176.2301
Epoch 589/10000
1/1 [==============================] - 0s 3ms/step - loss: 176.2088
Epoch 590/10000
1/1 [==============================] - 0s 2ms/step - loss: 176.2367
Epoch 591/10000
1/1 [==============================] - 0s 3ms/step - loss: 176.1798
Epoch 592/10000
1/1 [==============================] - 0s 2ms/step - loss: 176.1779
Epoch 593/10000
1/1 [==============================] - 0s 2ms/step - loss: 176.1779
Epoch 594/10000
1/1 [==============================] - 0s 2ms/step - loss: 176.1390
Epoch 595/10000
1/1 [==============================] - 0s 3ms/step - loss: 176.1438
Epoch 59

1/1 [==============================] - 0s 2ms/step - loss: 175.0296
Epoch 779/10000
1/1 [==============================] - 0s 2ms/step - loss: 175.0296
Epoch 780/10000
1/1 [==============================] - 0s 2ms/step - loss: 175.0377
Epoch 781/10000
1/1 [==============================] - 0s 2ms/step - loss: 175.0246
Epoch 782/10000
1/1 [==============================] - 0s 2ms/step - loss: 175.0160
Epoch 783/10000
1/1 [==============================] - 0s 3ms/step - loss: 175.0040
Epoch 784/10000
1/1 [==============================] - 0s 2ms/step - loss: 175.0020
Epoch 785/10000
1/1 [==============================] - 0s 2ms/step - loss: 175.0005
Epoch 786/10000
1/1 [==============================] - 0s 2ms/step - loss: 174.9977
Epoch 787/10000
1/1 [==============================] - 0s 2ms/step - loss: 174.9955
Epoch 788/10000
1/1 [==============================] - 0s 2ms/step - loss: 174.9901
Epoch 789/10000
1/1 [==============================] - 0s 2ms/step - loss: 174.9865
Epoch 79

1/1 [==============================] - 0s 2ms/step - loss: 174.2319
Epoch 973/10000
1/1 [==============================] - 0s 2ms/step - loss: 174.2369
Epoch 974/10000
1/1 [==============================] - 0s 2ms/step - loss: 174.2301
Epoch 975/10000
1/1 [==============================] - 0s 2ms/step - loss: 174.2219
Epoch 976/10000
1/1 [==============================] - 0s 2ms/step - loss: 174.2231
Epoch 977/10000
1/1 [==============================] - 0s 2ms/step - loss: 174.2214
Epoch 978/10000
1/1 [==============================] - 0s 2ms/step - loss: 174.2134
Epoch 979/10000
1/1 [==============================] - 0s 2ms/step - loss: 174.2080
Epoch 980/10000
1/1 [==============================] - 0s 2ms/step - loss: 174.2133
Epoch 981/10000
1/1 [==============================] - 0s 2ms/step - loss: 174.2230
Epoch 982/10000
1/1 [==============================] - 0s 2ms/step - loss: 174.1977
Epoch 983/10000
1/1 [==============================] - 0s 2ms/step - loss: 174.2461
Epoch 98

1/1 [==============================] - 0s 2ms/step - loss: 173.6184
Epoch 1165/10000
1/1 [==============================] - 0s 2ms/step - loss: 173.6153
Epoch 1166/10000
1/1 [==============================] - 0s 2ms/step - loss: 173.6172
Epoch 1167/10000
1/1 [==============================] - 0s 2ms/step - loss: 173.6040
Epoch 1168/10000
1/1 [==============================] - 0s 2ms/step - loss: 173.6027
Epoch 1169/10000
1/1 [==============================] - 0s 2ms/step - loss: 173.5953
Epoch 1170/10000
1/1 [==============================] - 0s 2ms/step - loss: 173.5923
Epoch 1171/10000
1/1 [==============================] - 0s 2ms/step - loss: 173.5887
Epoch 1172/10000
1/1 [==============================] - 0s 2ms/step - loss: 173.5839
Epoch 1173/10000
1/1 [==============================] - 0s 2ms/step - loss: 173.5842
Epoch 1174/10000
1/1 [==============================] - 0s 3ms/step - loss: 173.5778
Epoch 1175/10000
1/1 [==============================] - 0s 3ms/step - loss: 173.57

1/1 [==============================] - 0s 2ms/step - loss: 172.6473
Epoch 1357/10000
1/1 [==============================] - 0s 2ms/step - loss: 172.5835
Epoch 1358/10000
1/1 [==============================] - 0s 3ms/step - loss: 172.4717
Epoch 1359/10000
1/1 [==============================] - 0s 2ms/step - loss: 171.7249
Epoch 1360/10000
1/1 [==============================] - 0s 2ms/step - loss: 180.0587
Epoch 1361/10000
1/1 [==============================] - 0s 2ms/step - loss: 172.4196
Epoch 1362/10000
1/1 [==============================] - 0s 2ms/step - loss: 172.6641
Epoch 1363/10000
1/1 [==============================] - 0s 2ms/step - loss: 172.9854
Epoch 1364/10000
1/1 [==============================] - 0s 2ms/step - loss: 173.1545
Epoch 1365/10000
1/1 [==============================] - 0s 2ms/step - loss: 173.0444
Epoch 1366/10000
1/1 [==============================] - 0s 2ms/step - loss: 173.1151
Epoch 1367/10000
1/1 [==============================] - 0s 3ms/step - loss: 173.11

In [30]:
from math import sqrt
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score


yhat = model.predict(X_test)
rmse = sqrt(mean_squared_error(y_test, yhat))
r2 = r2_score(y_test, yhat)

print(rmse)
print(r2)


1/1 [==============================] - 0s 12ms/step
318.55405418503295
-0.26317629576877044
